In [ ]:
import psycopg2 as pg2
import pandas as pd
import math

In [ ]:
print ("Connecting to SQL")
conn = pg2.connect(user='postgres',  dbname='penny', host='localhost', port='5432', password='')

print ("Reading Dataset")
df = pd.read_sql ("""Select * from auction_full where auction_date >= """, conn)

In [ ]:
df.perc_to_bin[(df.perc_to_bin > .1 ) & (df.perc_to_bin < 1)&(df.giveup)].hist(bins=20)

In [1]:
df.perc_to_bin[(df.perc_to_bin > .1 ) & (df.perc_to_bin < 1)&(df.giveup==False)].hist(bins=20)


NameError: name 'df' is not defined

In [ ]:
df["bucket"]=df.perc_to_bin.apply(lambda x: int(round(int(x*300))/30.0))

In [ ]:
df.groupby("bucket").mean().loc[:,["bucket", "giveup"]].head(11).plot()

In [ ]:
df.groupby("bucket").mean().loc[:,["perc_to_bin", "giveup"]].head(11)

In [ ]:
conn.close()